In [61]:
import pandas as pd
import geopandas as gpd
import requests

In [64]:
def get_data(data_url: str, cbs = False):
    """Function to retrieve datasets from the internet

    Parameters
    ----------
    data_url : string
        The url that points to the dataset

    cbs: bool
        Whether the data is hosted by CBS

    Returns
    -------
    pd.DataFrame
        Dataframe containing data retrieved from data_url
    """    
    data = pd.DataFrame()
    if cbs:
        while data_url:
            r = requests.get(data_url).json()
            data = data.append(pd.DataFrame(r['value']))
            
            if '@odata.nextLink' in r:
                data_url = r['@odata.nextLink']
            else:
                data_url = None
    else:
        r = requests.get(data_url).json()
        data = data.append(pd.DataFrame(r))
    return data

In [65]:
covid19_link = 'https://data.rivm.nl/covid-19/COVID-19_aantallen_gemeente_cumulatief.json'
cbs_link = "https://beta-odata4.cbs.nl/CBS/70072ned"


In [6]:
covid19_df = get_data(covid19_link)
covid19_df.to_csv('Data/COVID_19_cumulatief.csv')

In [69]:
# Filter to only download municipality data
cbs_kerncijfers = get_data(f"{cbs_link}/Observations?$filter=startswith(RegioS, 'GM') and startswith(Perioden, '202')", cbs=True)

# Collect CBS metadata
groups = get_data(cbs_link + "/MeasureGroups", cbs = True)
codes = get_data(cbs_link + "/MeasureCodes", cbs=True)

In [68]:
cbs_kerncijfers

,Id,Measure,ValueAttribute,Value,RegioS,Perioden
0,375789,M000352_3,Impossible,NaN,GM1680,1995JJ00
1,375790,3000,Impossible,NaN,GM1680,1995JJ00
2,375791,4000,Impossible,NaN,GM1680,1995JJ00
3,375792,40000_1,Impossible,NaN,GM1680,1995JJ00
4,375793,70200_1,Impossible,NaN,GM1680,1995JJ00
...,...,...,...,...,...,...
6998,5282787,A045820_3,Impossible,NaN,GM0193,2021JJ00
6999,5282788,A045823_3,Impossible,NaN,GM0193,2021JJ00
7000,5282789,GM000A,Impossible,NaN,GM0193,2021JJ00
7001,5282790,M000161,None,16.0,GM0193,2021JJ00


In [67]:
# Ensure that we only select municipalities
cbs_kerncijfers = cbs_kerncijfers[cbs_kerncijfers['RegioS'].str.contains('GM')]
# Only select data from 2020 or 2021
cbs_kerncijfers = cbs_kerncijfers[cbs_kerncijfers['Perioden'].str.contains('202')]

# Add metadata for codes
cbs_kerncijfers_meta = pd.merge(cbs_kerncijfers, codes, left_on="Measure",right_on="Identifier")

In [99]:
# Pivot van wide naar long 
cbs_kerncijfers_meta_pivot = cbs_kerncijfers_meta.pivot(index=['RegioS','Perioden'],columns=['Identifier','Title','Description'],values='Value')

In [100]:
cbs_kerncijfers_meta_pivot.to_csv('Data/CBS_kerncijfers_pivot.csv')

# Geodata inlezen

# Lezen en combineren data

In [4]:
covid19_df = pd.read_csv('Data/COVID_19_cumulatief.csv')

In [26]:
cbs_kerncijfers_meta_pivot = pd.read_csv('Data/CBS_kerncijfers_pivot.csv',header=[1])
cbs_kerncijfers_meta_pivot = cbs_kerncijfers_meta_pivot.drop([0,1])

In [55]:
cbs_kerncijfers_meta_pivot.columns[200:250]

Index(['Bedrijfsmotorvoertuigen', 'Motorfietsen', 'Motorfietsen, relatief',
       'Voertuigen met bromfietskenteken',
       'Voertuigen met bromfietskenteken (%)', 'Totale weglengte',
       'Gemeentelijke en waterschapswegen', 'Provinciale wegen', 'Rijkswegen',
       'Afstand tot kinderdagverblijf',
       'Aantal kinderdagverblijven binnen 3 km',
       'Afstand tot grote supermarkt', 'Aantal grote supermarkten binnen 3 km',
       'Afstand tot bibliotheek', 'Afstand tot bioscoop',
       'Aantal bioscopen binnen 10 km', 'Afstand tot oprit hoofdverkeersweg',
       'Totaal huishoudelijk afval', 'Huishoudelijk restafval',
       'Grof huishoudelijk restafval', 'Gft-afval', 'Oud papier en karton',
       'Verpakkingsglas', 'Textiel', 'Klein chemisch afval',
       'Overig huishoudelijk afval', 'Totale oppervlakte.1', 'Land',
       'Water totaal', 'Binnenwater', 'Buitenwater',
       'Omgevingsadressendichtheid', 'Verkeersterrein', 'Bebouwd terrein',
       'Semi-bebouwd terrein', '

In [56]:
cbs_kerncijfers_klein = cbs_kerncijfers_meta_pivot[['Title','Totale bevolking','Mannen','Vrouwen','Jonger dan 5 jaar.1',
       '5 tot 10 jaar.1', '10 tot 15 jaar.1', '15 tot 20 jaar.1',
       '20 tot 25 jaar.1', '25 tot 45 jaar.1', '45 tot 65 jaar.1',
       '65 tot 80 jaar.1', '80 jaar of ouder.1','Nederlandse achtergrond.1',
       'Totaal met migratieachtergrond.1', 'Westerse migratieachtergrond.1',
       'Totaal niet-westerse migratieachtergrond.1', 'Marokko.1',
       '(voormalige) Nederlandse Antillen, Aruba.1', 'Suriname.1', 'Turkije.1',
       'Overig niet-westerse migratieachtergrond.1','Sterfte', 'Sterfte, relatief', 'Bevolkingsdichtheid',
       'Woningdichtheid', 'Koopwoningen', 'Huurwoningen']]

In [57]:
covid19_df_totalen = covid19_df[covid19_df['Date_of_report']=='2021-11-10 10:00:00']

In [58]:
result_df_klein = covid19_df_totalen.merge(cbs_kerncijfers_klein,how='left',left_on='Municipality_code',right_on=('Title'))

In [60]:
result_df_klein.dropna().to_csv('Data/Combi_df_klein.csv')